In [1]:
from tcn import TCN
import numpy as np

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K

In [2]:
X = np.load('X1.npy')
y = np.load('y1.npy')

In [3]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [4]:
def compile_model(swp_X, nb_filters, kernel_size, dilations, dropout_rate, y_train):

    # output tcn2
    nb_features = swp_X.shape[2]
    nb_outputs = y_train.shape[1]

    i = Input(shape=(swp_X.shape[1], nb_features))

    m = TCN(nb_filters=nb_filters, kernel_size=kernel_size, dropout_rate=dropout_rate, dilations=dilations, return_sequences=True, activation='relu')(i)
    m = TCN(nb_filters=nb_filters, dropout_rate=dropout_rate, kernel_size=kernel_size, dilations=dilations, return_sequences=False, activation='relu')(m)
    m = Dense(nb_outputs, activation='linear')(m)
    model = Model(inputs=[i], outputs=[m])
    model.compile(loss=root_mean_squared_error,
              optimizer='adam', metrics=['mae', root_mean_squared_error])

    return model

In [5]:
model = compile_model(X, nb_filters=64, kernel_size=3, dilations=[1, 2, 4, 8, 16, 32, 64], dropout_rate=0.05, y_train=y)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 42, 5)]           0         
                                                                 
 tcn (TCN)                   (None, 42, 64)            161984    
                                                                 
 tcn_1 (TCN)                 (None, 64)                172928    
                                                                 
 dense (Dense)               (None, 4)                 260       
                                                                 
Total params: 335,172
Trainable params: 335,172
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
150/150 [==============================] - 7s 44ms/step - loss: 4.6764 - mae: 3.2245 - root_mean_squared_error: 4.6757 - val_loss: 8.0544 - val_mae: 5.7409 - val_root_mean_squared_error: 8.0400
Epoch 2/10
150/150 [==============================] - 6s 41ms/step - loss: 4.2760 - mae: 2.9690 - root_mean_squared_error: 4.2756 - val_loss: 7.3312 - val_mae: 5.2281 - val_root_mean_squared_error: 7.3287
Epoch 3/10
150/150 [==============================] - 6s 43ms/step - loss: 3.8394 - mae: 2.7091 - root_mean_squared_error: 3.8393 - val_loss: 6.6147 - val_mae: 4.8163 - val_root_mean_squared_error: 6.6062
Epoch 4/10
150/150 [==============================] - 6s 43ms/step - loss: 3.8372 - mae: 2.6940 - root_mean_squared_error: 3.8372 - val_loss: 8.9754 - val_mae: 6.1831 - val_root_mean_squared_error: 8.9547
Epoch 5/10
150/150 [==============================] - 7s 43ms/step - loss: 3.7690 - mae: 2.6464 - root_mean_squared_error: 3.7693 - val_loss: 9.0647 - val_mae: 6.1403 - val_root_me